# Notebook for the Week 03 Assignament of the Coursera Applied Data Science Capstone Project
--> Segmenting and Clustering Neighborhoods in Toronto <--
PART 01

Download all the libraries we will need

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium=0.5.0 --yes  
import folium # map rendering library

# libraries for web scraping
%pip install beautifulsoup4   
from bs4 import BeautifulSoup
import urllib.request

print('<<<<<<<<<<<<<<<<<<< Libraries imported >>>>>>>>>>>>>>>>>>>')

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Note: you may need to restart the kernel to use updated packages.
<<<<<<<<<<<<<<<<<<< Libraries imported >>>>>>>>>>>>>>>>>>>


First, we define the wikipedia page to get the data about Toronto and extract the relevant table

In [2]:
# url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"  # new version of the page
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=862527922"# original version

# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

# show the parsed hmtl in soup object
# print(soup.prettify())

# <table class="wikitable sortable"> <-- this is the table we are looking for
data_table=soup.find('table', class_='wikitable sortable')
data_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

Once we get our desired table, we clean it in order to get just the relevant data, without html code

In [3]:
A=[]
B=[]
C=[]

for row in data_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))        

#### Question 3.5 
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
for i, cell3 in enumerate(C):
    if cell3 == 'Not assigned\n':
        C[i] = B[i]
# print(C)

#### Question 3.1 
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [5]:
import pandas as pd
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C

df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
9,M8A,Not assigned,Not assigned


In [6]:
# remove final '\n' string from every record
df['PostalCode'] = df['PostalCode'].str.replace(r'\n', '')
df['Borough'] = df['Borough'].str.replace(r'\n', '')
df['Neighborhood'] = df['Neighborhood'].str.replace(r'\n', '')    


#### Question 3.4 
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [13]:
df = df[df['Borough']!= 'Not assigned']
df.to_excel("resultado_3_3.xlsx")
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### Question 3.3 
More than one neighborhood can exist in one postal code area. 

In [14]:
df_final = df.groupby(["PostalCode","Borough"])["Neighborhood"].apply(lambda values: ','.join(values)).reset_index(name='Neighborhood')

In [15]:
df_final

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [16]:
# send the final result to excel for testing
df_final.to_excel("resultado_final.xlsx")

#### Question 3.6
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [17]:
df_final.shape

(103, 3)